<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1Predict_by_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret==2.3.1

In [3]:
from pycaret.utils import enable_colab

enable_colab()

Colab mode enabled.


In [77]:
import numpy as np
import pandas as pd
import time

csv = '/content/seasonData_2021.csv'
data = pd.read_csv(csv)
data = data.drop(data.columns[[0,7,18]], axis=1)
data.shape

(380, 26)

In [78]:
data.to_csv("seasonData2021.csv",encoding = 'utf-8-sig')

# データの加工

In [34]:
data['CBP_x'] = data['CBP_x'].str.replace('%', '').astype(np.float64) /100
data['shot acc_x'] = data['shot acc_x'].str.replace('%', '').astype(np.float64) / 100
data['possession_x'] = data['possession_x'].str.replace('%', '').astype(np.float64) / 100
data['CBP_y'] = data['CBP_y'].str.replace('%', '').astype(np.float64) / 100
data['shot acc_y'] = data['shot acc_y'].str.replace('%', '').astype(np.float64) / 100
data['possession_y'] = data['possession_y'].str.replace('%', '').astype(np.float64) / 100

# pandas profiling

In [ ]:
!pip install --upgrade pandas-profiling

In [ ]:
import pandas_profiling as pdp
pdp.ProfileReport(data).to_file(output_file = 'profileReport.html')

# SetUP

In [5]:
!pip install pyyaml==5.4.1

In [35]:
from pycaret.regression import setup
data = data.drop('節', axis=1)
data = data.drop('開催日', axis=1)
def x(a,b):
    return a - b
data['homeScore']  = data.apply(lambda f: x(f['homeScore'],f['awayScore']), axis=1)
data = data.drop('awayScore', axis=1)
data = data.rename(columns={'homeScore': 'Score'})
data = data.drop('H or A_x',axis = 1)
data = data.drop('H or A_y',axis = 1)
data

,Team,awayTeam,Score,AGI_x,KAGI_x,CBP_x,shot_x,shot acc_x,possession_x,ACBP_x,PassCBP_x,InterceptP_x,DP_x,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
0,広島,仙台,0,70,59,0.176,19,0.053,0.576,23.40,19.60,83.02,10.45,41,30,0.061,7,0.143,0.424,9.07,7.21,82.69,17.97
1,神戸,Ｇ大阪,1,12,51,0.063,8,0.125,0.582,11.54,10.22,118.05,13.65,49,88,0.061,8,0.000,0.418,12.28,10.08,101.88,12.44
2,大分,徳島,0,42,49,0.094,10,0.100,0.374,8.78,6.79,93.51,18.91,51,58,0.126,13,0.077,0.626,17.04,13.42,99.86,10.39
3,福岡,名古屋,-1,50,41,0.033,4,0.000,0.505,13.11,10.11,113.83,11.57,59,50,0.081,10,0.200,0.495,13.29,10.12,105.37,14.75
4,川崎Ｆ,横浜FM,2,57,44,0.137,18,0.111,0.469,15.98,14.30,139.34,13.68,56,43,0.082,11,0.000,0.531,17.02,15.60,108.41,10.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,清水,Ｃ大阪,1,59,66,0.190,20,0.100,0.389,10.59,8.16,86.22,14.27,34,41,0.061,6,0.000,0.611,14.20,13.65,46.62,13.03
376,名古屋,浦和,0,41,51,0.081,9,0.000,0.437,10.72,9.43,97.49,14.56,49,59,0.075,8,0.000,0.563,17.25,14.37,99.61,9.07
377,横浜FM,川崎Ｆ,0,57,37,0.150,20,0.050,0.579,24.42,21.26,95.87,14.83,63,43,0.094,12,0.083,0.421,16.68,11.61,134.99,18.46
378,仙台,鹿島,-1,49,37,0.090,11,0.000,0.436,11.92,9.39,72.12,15.70,63,51,0.131,16,0.063,0.564,17.35,14.31,84.47,15.27


In [32]:
setup = setup(data, target="Score",
        session_id = 123)

TypeError: ignored

In [31]:
setup

([('Setup Config',                                Description             Value
   0                               session_id               123
   1                                   Target             Score
   2                            Original Data         (380, 23)
   3                           Missing Values             False
   4                         Numeric Features                20
   5                     Categorical Features                 2
   6                         Ordinal Features             False
   7                High Cardinality Features             False
   8                  High Cardinality Method              None
   9                    Transformed Train Set         (265, 57)
   10                    Transformed Test Set         (115, 57)
   11                      Shuffle Train-Test              True
   12                     Stratify Train-Test             False
   13                          Fold Generator             KFold
   14                   

In [9]:
from pycaret.regression import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,4.286000e-01,4.196000e-01,6.283000e-01,8.650000e-01,0.0,2.767000e-01,0.122
rf,Random Forest Regressor,4.894000e-01,5.894000e-01,7.409000e-01,8.141000e-01,0.0,3.062000e-01,0.664
et,Extra Trees Regressor,4.708000e-01,5.652000e-01,7.353000e-01,8.133000e-01,0.0,2.959000e-01,0.483
omp,Orthogonal Matching Pursuit,5.341000e-01,6.321000e-01,7.704000e-01,7.885000e-01,0.0,3.541000e-01,0.027
lr,Linear Regression,6.225000e-01,7.031000e-01,8.219000e-01,7.616000e-01,0.0,4.112000e-01,0.702
br,Bayesian Ridge,6.365000e-01,7.094000e-01,8.295000e-01,7.585000e-01,0.0,4.143000e-01,0.027
ada,AdaBoost Regressor,6.659000e-01,8.281000e-01,8.930000e-01,7.354000e-01,0.0,4.310000e-01,0.113
ridge,Ridge Regression,7.028000e-01,8.277000e-01,8.993000e-01,7.159000e-01,0.0,4.474000e-01,0.027
dt,Decision Tree Regressor,6.905000e-01,1.225500e+00,1.098400e+00,5.791000e-01,0.0,4.577000e-01,0.514
huber,Huber Regressor,1.034100e+00,1.866900e+00,1.348900e+00,3.690000e-01,0.0,6.421000e-01,0.074


In [10]:
print(best_model)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


# Create Model

In [11]:
from pycaret.regression import models

models()  # 機械学習モデルの一覧

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [12]:
from pycaret.regression import create_model

gbr = create_model("gbr")  # GradientBoostingRegressorのモデルを作成

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.2929,0.2008,0.4481,0.9014,-0.0,0.2379
1,0.3655,0.2718,0.5213,0.8741,-0.0,0.2704
2,0.4809,0.4955,0.7039,0.8454,-0.0,0.2683
3,0.3459,0.1859,0.4312,0.9388,-0.0,0.2323
4,0.6178,0.9171,0.9576,0.7878,-0.0,0.2770
5,0.3941,0.3250,0.5701,0.8868,-0.0,0.2963
6,0.3438,0.2737,0.5231,0.8697,-0.0,0.2586
7,0.4985,0.5911,0.7688,0.7605,-0.0,0.3326
8,0.5007,0.5702,0.7551,0.8581,-0.0,0.3512
9,0.4456,0.3647,0.6039,0.9276,-0.0,0.2420


In [13]:
print(gbr)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


# Tune Model

In [14]:
from pycaret.regression import tune_model

tuned_gbr = tune_model(gbr)  # ハイパーパラメータの調整

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.3262,0.2274,0.4769,0.8883,-0.0,0.2848
1,0.3692,0.2962,0.5442,0.8628,-0.0,0.3018
2,0.4171,0.3109,0.5576,0.9030,-0.0,0.2336
3,0.3564,0.1882,0.4338,0.9380,-0.0,0.2547
4,0.5347,0.7720,0.8786,0.8213,-0.0,0.2506
5,0.3627,0.2509,0.5009,0.9126,-0.0,0.3013
6,0.4535,0.5397,0.7346,0.7431,-0.0,0.2717
7,0.4634,0.4980,0.7057,0.7982,-0.0,0.2891
8,0.5400,0.7274,0.8529,0.8190,-0.0,0.3645
9,0.4788,0.4134,0.6429,0.9179,-0.0,0.2720


In [15]:
print(tuned_gbr)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.15, loss='ls', max_depth=7,
                          max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.02, min_impurity_split=None,
                          min_samples_leaf=5, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=230,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=0.85, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


# evalation

In [16]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_gbr)

# deploy

In [17]:
from pycaret.regression import finalize_model

final_gbr = finalize_model(tuned_gbr)
print(final_gbr)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.15, loss='ls', max_depth=7,
                          max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.02, min_impurity_split=None,
                          min_samples_leaf=5, min_samples_split=5,
                          min_weight_fraction_leaf=0.0, n_estimators=230,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=123, subsample=0.85, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


In [18]:
from pycaret.regression import predict_model

predict_model(final_gbr);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,0.0357,0.0025,0.0496,0.999,0.0264,0.0263


In [19]:
from pycaret.regression import save_model

save_model(final_gbr, "gbr_model")

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='Score',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='...
                                            learning_rate=0.15, loss='ls',
                                            max_depth=7, max_features=1.0,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.02,
                          

In [20]:
from pycaret.regression import load_model

loaded_gbr = load_model("gbr_model")

Transformation Pipeline and Model Successfully Loaded


# get Pred Data

In [83]:
def scrapePredData(homeTeam,awayTeam,TeamNameDic,year):
    team = TeamNameDic[homeTeam]
    team2 = TeamNameDic[awayTeam]
    url = 'https://www.football-lab.jp/' +  team + '/match/' + '?year=' + year
    url2 = 'https://www.football-lab.jp/' +  team2 + '/match/' + '?year=' + year
    #試合結果を取得
    #表を取得
    df = pd.read_html(url)[0]
    df2 = pd.read_html(url2)[0]
    #最後の列を削除
    last_index = df.index[-1]
    last_index2 = df2.index[-1]
    df = df.drop(last_index).drop(df.columns[[2,6,7,8,19,20]], axis=1)
    df2 = df2.drop(last_index2).drop(df2.columns[[2,6,7,8,19,20]], axis=1)
    #homeTeam名追加
    df.insert(2,'Team',homeTeam)
    df2.insert(2,'Team',awayTeam)
    #列名を英語に
    df = df.rename(columns={'相手': 'awayTeam','Unnamed: 5':'H or A','チャンス構築率':'CBP_x','シュート':'shot_x','シュート成功率':'shot acc_x','支配率':'possession_x','攻撃CBP':'ACBP_x','パスCBP':'PassCBP_x','奪取P':'InterceptP_x','守備P':'DP_x'})
    df2 = df2.rename(columns={'相手': 'awayTeam','Unnamed: 5':'H or A','チャンス構築率':'CBP_y','シュート':'shot_y','シュート成功率':'shot acc_y','支配率':'possession_y','攻撃CBP':'ACBP_y','パスCBP':'PassCBP_y','奪取P':'InterceptP_y','守備P':'DP_y'})
    
    #%を変換
    df['CBP_x'] = df['CBP_x'].str.replace('%', '').astype(np.float64) /100
    df['shot acc_x'] = df['shot acc_x'].str.replace('%', '').astype(np.float64) / 100
    df['possession_x'] = df['possession_x'].str.replace('%', '').astype(np.float64) / 100
    df2['CBP_y'] = df2['CBP_y'].str.replace('%', '').astype(np.float64) /100
    df2['shot acc_y'] = df2['shot acc_y'].str.replace('%', '').astype(np.float64) / 100
    df2['possession_y'] = df2['possession_y'].str.replace('%', '').astype(np.float64) / 100

    #一旦書き出して数値化して、平均をとる
    df.to_csv("df.csv",encoding = 'utf-8-sig')
    data = pd.read_csv('/content/df.csv')
    data = data.drop(data.columns[[0]], axis=1)
    data.loc[0] =data.mean()
    df2.to_csv("df2.csv",encoding = 'utf-8-sig')
    data2 = pd.read_csv('/content/df2.csv')
    data2 = data2.drop(data2.columns[[0]], axis=1)
    data2.loc[0] =data2.mean()

    pred = pd.DataFrame(data[:1])
    pred = pred.drop(data.columns[[0,1,4,5]], axis=1)
    pred['Team'] = homeTeam
    pred['awayTeam'] = awayTeam
    pred = pred.rename(columns={'AGI': 'AGI_x','KAGI': 'KAGI_x'})

    pred2 = pd.DataFrame(data2[:1])
    pred2 = pred2.drop(data2.columns[[0,1,2,3,4,5]], axis=1)
    pred2 = pred2.rename(columns={'AGI': 'AGI_y','KAGI': 'KAGI_y'})
    pred = pd.concat([pred, pred2], axis=1)
    return pred


### Team Dic

In [87]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}
J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}
J12022TeamDic = {'札幌':'sapp', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm' , '湘南':'shon', '清水':'shim', '磐田':'iwat','名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '京都':'kyot','神戸':'kobe','広島':'hiro', '福岡':'fuku', '鳥栖':'tosu'}
J22022TeamDic = {'岩手':'iwte','仙台':'send','秋田':'aki','山形':'yama','水戸':'mito','栃木':'toch','群馬':'gun','大宮':'omiy','千葉':'chib','東京V':'tk-v','町田':'mcd','横浜FC':'y-fc','甲府':'kofu','新潟':'niig','金沢':'kana','岡山':'okay','山口':'r-ya','徳島':'toku','長崎':'ngsk','熊本':'kuma','大分':'oita','琉球':'ryuk'}
def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

In [ ]:
predData = scrapePredData('川崎Ｆ','FC東京',J12021TeamDic,'2021')
predData

# prediction

In [84]:
def prediction(homeTeam,awayTeam,TeamDic,year,model):
    predData = scrapePredData(homeTeam,awayTeam,TeamDic,year)
    prediction = predict_model(model, data=predData)
    print(homeTeam,'vs' ,awayTeam , ':' , prediction['Label'][0])
    return 

### ここのTeamを変えれば予想できる

In [58]:
prediction('川崎Ｆ','FC東京', J12022TeamDic ,'2021' , loaded_gbr)

川崎Ｆ vs FC東京 : 1.0989408550798692


In [67]:
prediction('京都','浦和', J12022TeamDic ,'2021' , loaded_gbr)

京都 vs 浦和 : 0.3939028201120569


In [60]:
prediction('広島','鳥栖', J12022TeamDic ,'2021' , loaded_gbr)

広島 vs 鳥栖 : 0.04645790925594323


In [61]:
prediction('清水','札幌', J12022TeamDic ,'2021' , loaded_gbr)

清水 vs 札幌 : -0.032217416960376416


In [68]:
prediction('福岡','磐田', J12022TeamDic ,'2021' , loaded_gbr)

福岡 vs 磐田 : -0.29523586716353034


In [69]:
prediction('横浜FM','Ｃ大阪', J12022TeamDic ,'2021' , loaded_gbr)

横浜FM vs Ｃ大阪 : 0.881853961893402


In [70]:
prediction('Ｇ大阪','鹿島', J12022TeamDic ,'2021' , loaded_gbr)

Ｇ大阪 vs 鹿島 : -0.3269533278380414


In [71]:
prediction('名古屋','神戸', J12022TeamDic ,'2021' , loaded_gbr)

名古屋 vs 神戸 : -0.3926207258419101


In [72]:
prediction('湘南','柏', J12022TeamDic ,'2021' , loaded_gbr)

湘南 vs 柏 : -0.033364975789482165


In [85]:
prediction('長崎','東京V', J22022TeamDic ,'2021' , loaded_gbr)

長崎 vs 東京V : 0.5620813763095822


In [88]:
prediction('群馬','山形', J22022TeamDic ,'2021' , loaded_gbr)

群馬 vs 山形 : -0.18873081044279416


In [89]:
prediction('大分','水戸', J22022TeamDic ,'2021' , loaded_gbr)

大分 vs 水戸 : -0.35244147427270983


In [90]:
prediction('千葉','岩手', J22022TeamDic ,'2021' , loaded_gbr)

千葉 vs 岩手 : 0.03362511019014442
